In [1]:
import numpy as np
import os
import random as r
import test_multiprocess as tm
files_dir = r'C:\Users\roumba\Documents\Software\deep-events\training_data'
images_dir = '230201_mtStayGold_both_iSIMdestriped_fl_pos'
images_neg_dir = '230201_mtStayGold_both_iSIMdestriped_fl_neg'
import multiprocessing
sigmas = [2,3,4,5,6]
number_of_augmentations = 15
box_size=15
offset=3
background=0.1 # for fluorescent set this to 0.1 and for brightfield 0.05

for sigma in sigmas:
    print('sigma =',sigma)
    p = multiprocessing.Process(target=tm.load_aug_train, args=(files_dir, images_dir, images_neg_dir, sigma, number_of_augmentations, box_size, offset, background)) 
    p.start() 
    p.join()


# C:/Users/roumba/Documents/Software/deep-events/Modelsref_f8_c9_b16/model_both_ZEISS_fl_new1/model_both_ZEISS_fl_3.h5
# C:/Users/roumba/Documents/Software/deep-events/Modelsref_f8_c9_b16/model_cos7_ZEISS_bf_3.h5
# C:\Users\roumba\Documents\Software\deep-events\Modelsref_f8_c9_b16\model_cos7_ZEISS_bf_3.h5


### HOW TO ACTIVATE .ENV ###
# C:\Users\roumba\Documents\Software\deep-events\.env\Scripts\Activate.ps1


KeyboardInterrupt: 

In [5]:
import numpy as np
from glob import glob
from sklearn.model_selection import train_test_split
from mitosplit_net import util
from training_functions import create_model,train_model
from albumentations import Compose, Rotate, RandomRotate90, HorizontalFlip, Flip, VerticalFlip
import os
from os import path
import random as r
import tifffile
from import_augmentation_function import import_fun, aug_fun, import_fun_neg, normalization_fun_loc, normalization_fun_g, import_aug_fun
import tensorflow as tf

data_ratio= 0.1
data_split_state = None
BASE_DIR = r'C:\Users\roumba\Documents\Software\deep-events'


def load_aug_train(files_dir, images_dir,images_neg_dir, sigma, k,ofs,perc,savepath):
    date, dye, cell_type, microscope, bf_fl, pos_neg = images_dir.split('_')
    joined_path = os.path.join(files_dir, images_dir)
    joined_path_neg = os.path.join(files_dir, images_neg_dir)
    all_image_array, all_image_array_gauss= import_fun(joined_path, files_dir, images_dir,sigma)
    all_image_array_gauss, all_image_array= zero_frames(all_image_array.shape[1], all_image_array_gauss, all_image_array)
    all_image_array_neg, all_image_array_gauss_neg = import_fun_neg(joined_path_neg,files_dir, images_neg_dir)

    all_image_array = np.concatenate((all_image_array,all_image_array_neg))
    all_image_array_gauss = np.concatenate((all_image_array_gauss,all_image_array_gauss_neg))

 

    tifffile.imwrite(savepath +'\Pre_background_removal_fl.tiff', (all_image_array).astype(np.uint16), photometric='minisblack')
    tifffile.imwrite(savepath + '\Pre_background_removal_gauss_fl.tiff', (all_image_array_gauss).astype(np.uint8), photometric='minisblack')

    # imageio.mimwrite('Pre_background_removal.tiff', (all_image_array).astype(np.float64))
    # imageio.mimwrite('Pre_background_removal_gauss.tiff', (all_image_array_gauss).astype(np.float64))

                            ## NORMALIZATION ##

    norm_image_array= normalization_fun_loc(all_image_array, k, ofs, perc, bf_fl)
    norm_image_array_gauss = normalization_fun_g(all_image_array_gauss)

    tifffile.imwrite(savepath + '\Post_background_removal_fl.tiff', (norm_image_array).astype(np.float64), photometric='minisblack')
    tifffile.imwrite(savepath + '\Post_background_removal_gauss_fl.tiff', (norm_image_array_gauss).astype(np.float64), photometric='minisblack')

    # imageio.mimwrite('Post_background_removal.tiff', (norm_image_array).astype(np.float64))
    # imageio.mimwrite('Post_background_removal_gauss.tiff', (norm_image_array_gauss).astype(np.float64))

def zero_frames(size,datag,dataim):
    frame_check=np.zeros((size,size))
    z_list=[]

    for x in range(np.size(datag,0)):
        frame_check = datag[x]
        if not np.any(frame_check)==True:
            z_list.append(x)

    datag= np.delete(datag, z_list, axis=0)
    dataim= np.delete(dataim, z_list, axis=0)
    return datag,dataim


In [7]:
files_dir = r'C:\Users\roumba\Documents\Software\deep-events\training_data'
images_dir = '221123_mtStayGold_U2OS_ZEISS_fl_pos'
images_neg_dir = '221123_mtStayGold_U2OS_ZEISS_fl_neg'
sigma = 4
box_size=15
offset=3
background=0.1
load_aug_train(files_dir, images_dir, images_neg_dir, sigma, box_size, offset, background, BASE_DIR)

fluorescent removal
